In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Python 3.11.13


In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_path = "/content/drive/MyDrive/ML/Froud_Detation"

In [ ]:
print(image_path)
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

/content/drive/MyDrive/ML/Froud_Detation
['Fraud', 'Army_Uniform', 'Police_Uniform']


In [ ]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

In [ ]:
spec = image_classifier.SupportedModels.MOBILENET_V2
hparams = image_classifier.HParams(export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [ ]:
import os, shutil
from PIL import Image

# List of supported formats for TensorFlow
SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()   # Verify if it’s a valid image
        return True
    except Exception as e:
        print(f"Bad file: {file_path} - {e}")
        return False

# Clean dataset into a safe folder
filtered_dir = "/content/drive/MyDrive/ML/Froud_Detations"
if os.path.exists(filtered_dir):
    shutil.rmtree(filtered_dir)
os.makedirs(filtered_dir)

for label in labels:
    label_dir = os.path.join(image_path, label)
    dest_label_dir = os.path.join(filtered_dir, label)
    os.makedirs(dest_label_dir, exist_ok=True)
    for f in os.listdir(label_dir):
        file_path = os.path.join(label_dir, f)
        if any(f.lower().endswith(ext) for ext in SUPPORTED_EXTENSIONS) and is_valid_image(file_path):
            shutil.copy(file_path, dest_label_dir)  # Copy only clean images
        else:
            print(f"Skipping invalid/unsupported: {file_path}")

# Reload clean dataset
data = image_classifier.Dataset.from_folder(filtered_dir)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)


In [ ]:
import os, shutil
import tensorflow as tf
from PIL import Image

DATASET_DIR = "/content/drive/MyDrive/ML/Froud_Detations"   # your dataset path

bad_files = []

for root, dirs, files in os.walk(DATASET_DIR):
    for f in files:
        path = os.path.join(root, f)

        # only allow correct extensions
        if not f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif")):
            print("Deleting unsupported file:", path)
            os.remove(path)
            continue

        # try TensorFlow decode
        try:
            raw = tf.io.read_file(path)
            img = tf.image.decode_image(raw)
        except Exception as e:
            print("Bad file detected:", path, "-", e)
            bad_files.append(path)

# Try repairing bad files with PIL
for f in bad_files:
    try:
        img = Image.open(f).convert("RGB")
        new_path = f.rsplit(".", 1)[0] + "_fixed.jpg"
        img.save(new_path, "JPEG")
        os.remove(f)  # remove bad file
        print("Fixed and replaced:", f, "->", new_path)
    except Exception as e:
        print("Deleting permanently:", f, "-", e)
        os.remove(f)

print("Cleanup complete ✅")



Bad file detected: /content/drive/MyDrive/ML/Froud_Detations/Fraud/th.jpeg - {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage] name: 
Fixed and replaced: /content/drive/MyDrive/ML/Froud_Detations/Fraud/th.jpeg -> /content/drive/MyDrive/ML/Froud_Detations/Fraud/th_fixed.jpg
Cleanup complete ✅


In [ ]:
data = image_classifier.Dataset.from_folder(DATASET_DIR)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

model = image_classifier.ImageClassifier.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options,
)



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_7 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_7 (Dropout)         (None, 1280)              0         
                                                                 
 dense_7 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 3843 (15.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None
Epoch 1/10
60/60 [==============================] - 16s 151ms/step - loss: 1.6265 - accuracy: 0.3167 - val_loss: 1.0451 - val_accuracy: 0.3125
Epoch 2/10
60/60 [==============================] - 11s 177ms/step - loss: 1.4307 - accuracy: 0.3417 - val_loss: 0.9286 - val_accu

In [ ]:
hparams = image_classifier.HParams(epochs=30, export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

model = image_classifier.ImageClassifier.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options,
)


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_8 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_8 (Dropout)         (None, 1280)              0         
                                                                 
 dense_8 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 3843 (15.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None
Resuming from exported_model/checkpoint/model-0010
Epoch 1/30
60/60 [==============================] - 16s 166ms/step - loss: 0.8049 - accuracy: 0.6667 - val_loss: 0.6102 - val_accuracy: 0.8750
Epoch 2/30
60/60 [==============================] - 10s 158ms/step - loss: 0.86

In [ ]:
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc:.2f}")


1/1 [==============================] - 4s 4s/step - loss: 0.5728 - accuracy: 0.8667
Test Accuracy: 0.87


In [ ]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

1/1 [==============================] - 2s 2s/step - loss: 0.5728 - accuracy: 0.8667
Test loss:0.5728482007980347, Test accuracy:0.8666666746139526


In [ ]:
model.export_model()

In [ ]:
from google.colab import files
files.download("exported_model/model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install mediapipe-model-maker
!pip install mediapipe


In [ ]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Convert your TFLite to Task file
BaseOptions = python.BaseOptions
ImageClassifierOptions = vision.ImageClassifierOptions

options = ImageClassifierOptions(
    base_options=BaseOptions(model_asset_path="exported_model/model.tflite"),
    max_results=3
)

# Save as .task file
with open("exported_model/model.task", "wb") as f:
    f.write(open("exported_model/model.tflite", "rb").read())


In [ ]:
from google.colab import files

# Download the .task file
files.download("exported_model/model.task")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import core
from mediapipe.tasks.python import BaseOptions
import mediapipe as mp # Import mediapipe to access ImageFormat

# Load the model
options = vision.ImageClassifierOptions(
    base_options=BaseOptions(model_asset_path="exported_model/model.task"),
    max_results=3
)
classifier = vision.ImageClassifier.create_from_options(options)

# Read image
img = cv2.imread("/content/drive/MyDrive/Mca/IMG_20241004_210833.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert the NumPy array to a mediapipe.Image object
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

# Classify the image
result = classifier.classify(mp_image)

# Print results
for r in result.classifications[0].categories:
    print(f"Label: {r.category_name}, Confidence: {r.score:.2f}")

Label: Army_Uniform, Confidence: 0.55
Label: Police_Uniform, Confidence: 0.27
Label: Fraud, Confidence: 0.18
